Task 1: Logistic Regression

In [ ]:
import numpy as np
import pandas as pd

Sigmoid / Logistic Function

In [ ]:
'''
The sigmoid function squishes all input values between 0 and 1. It is used to convert the output of a linear function
into a probability, which is then used to make a binary decision. 

Parameters
----------
z : numpy array

Returns
-------
_ : numpy array
    the predicted output
'''

def sigmoid(z):
    
    return 1.0/(1 + np.exp(-z))

Loss Function

In [ ]:
'''
The loss function is the overall loss function for logistic regression for this specific iteration. 

Parameters
----------
y : numpy array
y_hat : numpy array

Returns
-------
_ : float
    the loss value of that particular iteration
'''

def loss(y, y_hat):
    
    return -np.mean(y*(np.log(y_hat)) - (1-y)*np.log(1-y_hat))

Gradient Descent

In [ ]:
'''
The gradients function calculates the partial derivatives of the loss function with respect to the weights and bias.
This partial derivative is used to update the weights and bias in the direction that minimizes the loss function.

Parameters
----------
X, inputs : numpy array
y, actual values : numpy array
y_hat, hypothesis/predictions : numpy array

Returns
-------
dw : float
    partial derivative of the loss function with respect to the weights
db : float
    partial derivative of the loss function with respect to the bias
'''

def gradients(X, y, y_hat):

    m = X.shape[0]
    
    dw = (1/m)*np.dot(X.T, (y_hat - y))
    db = (1/m)*np.sum((y_hat - y)) 
    
    return dw, db

Normalise

In [ ]:
'''
The normalise function normalises the input features by subtracting the mean and dividing by the standard deviation.
This helps to scale down the input features to a common scale, which helps in faster convergence of the gradient 
descent algorithm, and reduces the magnitude of the weights.

Parameters
----------
X, inputs : numpy array

Returns
-------
X : numpy array
    NumPy array of normalised input features
'''

def normalize(X):

    _, n = X.shape
    
    for i in range(n):
        X = (X - X.mean(axis=0))/X.std(axis=0)
        
    return X

Train Function

In [ ]:
'''
The train function trains the logistic regression model using the input features and target values. It uses the sigmoid 

Parameters
----------
X, inputs : numpy array
y, actual values : numpy array
bs, batch size : int
epochs, number of iterations : int
lr, learning rate : float

Returns
-------
w : numpy.ndarray
    The learned weights of the logistic regression model (shape: (n, 1)).
b : float
    The learned bias term of the logistic regression model.
losses : list of floats
    A list containing the loss values for each epoch during training.
'''

def train(X, y, bs, epochs, lr):
        
    # m-> number of training examples
    # n-> number of features 
    m, n = X.shape
    
    # Initializing weights and bias to zeros.
    w = np.zeros((n,1))
    b = 0
    
    # Reshaping y.
    y = y.reshape(m,1)
    
    # Normalizing the inputs.
    x = normalize(X)
    
    # Empty list to store losses.
    losses = []
    
    # Training loop.
    for _ in range(epochs):
        for i in range((m-1)//bs + 1):
            
            # Defining batches. SGD.
            start_i = i*bs
            end_i = start_i + bs
            xb = X[start_i:end_i]
            yb = y[start_i:end_i]
            
            # Calculating hypothesis/prediction.
            y_hat = sigmoid(np.dot(xb, w) + b)
            
            # Getting the gradients of loss w.r.t parameters.
            dw, db = gradients(xb, yb, y_hat)
            
            # Updating the parameters.
            w -= lr*dw
            b -= lr*db
        
        # Calculating loss and appending it in the list.
        l = loss(y, sigmoid(np.dot(X, w) + b))
        losses.append(l)
        
    # returning weights, bias and losses(List).
    return w, b, losses

Logistic Regression Main 

In [ ]:
# create variable w that is the weight. should be initialized to a zero vector of the same length as the number of features
# create variable b that is the bias. should be initialized to 0

# y_hat is the predicted value of y. It is calculated by taking the dot product of the weight vector, w and the feature vector X followed by the addition of the bias, b.
y_hat = sigmoid(w.X + b)